In [1]:
import os

In [2]:
%pwd

'c:\\Users\\prass\\OneDrive\\desktop\\practise\\new_env\\text-classification-using-BERT\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\prass\\OneDrive\\desktop\\practise\\new_env\\text-classification-using-BERT'

In [ ]:
import spacy
from nltk.corpus import stopwords
import string

# Load spaCy model
nlp = spacy.load('en_core_web_sm')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    
    text = text.lower() # Convert to lowercase

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize and lemmatize
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.text not in stop_words]
    return ' '.join(tokens)

# data['cleaned_text'] = data['text'].apply(preprocess_text)

In [7]:
from dataclasses import dataclass
from pathlib import Path



@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    params_num_train_epochs: int
    params_per_device_train_batch_size: int
    params_per_device_eval_batch_size: int
    params_warmup_steps: int
    params_weight_decay: float
    params_max_steps: int
    params_save_steps: int
    params_logging_steps: int

In [8]:
from textClassifier.constants import *
from textClassifier.utils.commons import read_yaml, create_directories

In [9]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])


    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            params_num_train_epochs=self.params.num_train_epochs,
            params_per_device_train_batch_size=self.params.per_device_train_batch_size,
            per_device_eval_batch_size=self.params.per_device_eval_batch_size,
            params_weight_decay=self.params.weight_decay,
            params_max_steps=self.params.max_steps,
            params_save_steps=self.params.save_steps,
            params_logging_steps=self.params.logging_steps



        )

        return prepare_base_model_config

In [19]:
import os
import urllib.request as request
from zipfile import ZipFile


In [11]:
import pandas as pd
# C:\Users\prass\OneDrive\desktop\practise\new_env\text-classification-using-BERT\artifacts\data_ingestion\Tweets.csv

data = pd.read_csv('c:\\Users\\prass\\OneDrive\\desktop\\practise\\new_env\\text-classification-using-BERT\\artifacts\data_ingestion\Tweets.csv') # Load the dataset

data.head() # Display the first few rows

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)


In [34]:
import spacy
from nltk.corpus import stopwords
import string

# Load spaCy model
nlp = spacy.load('en_core_web_sm')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    
    text = text.lower() # Convert to lowercase

    # Remove punctuation
    text = text.translate(str.maketrans('', '', string.punctuation))

    # Tokenize and lemmatize
    doc = nlp(text)
    tokens = [token.lemma_ for token in doc if token.text not in stop_words]
    return ' '.join(tokens)

# data['cleaned_text'] = data['text'].apply(preprocess_text)

def pass_data_to_text_process(data):
    data['cleaned_text'] = data['text'].apply(preprocess_text)
    return data

In [26]:
# label_mapping = {'negative': 0, 'neutral': 1, 'positive': 2}
# data['label'] = data['airline_sentiment'].map(label_mapping)

def mapping_data(data):
    label_mapping = {'negative': 0, 'neutral': 1, 'positive': 2}
    data['label'] = data['airline_sentiment'].map(label_mapping)
    return data



In [24]:
from transformers import BertTokenizer

# Tokenization
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Tokenize the data
def tokenize_function(text):
    return tokenizer(text, padding='max_length', truncation=True, max_length=128)

# data['tokenized'] = data['cleaned_text'].apply(tokenize_function)

def pass_data_to_tokenize(data):
    data['tokenized'] = data['cleaned_text'].apply(tokenize_function)
    return data

,tweet_id,airline_sentiment,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,cleaned_text,label,tokenized
0,570306133677760513,neutral,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),virginamerica dhepburn say,1,"[input_ids, token_type_ids, attention_mask]"
1,570301130888122368,positive,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),virginamerica plus add commercial experience t...,2,"[input_ids, token_type_ids, attention_mask]"
2,570301083672813571,neutral,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),virginamerica not today must mean need take an...,1,"[input_ids, token_type_ids, attention_mask]"
3,570301031407624196,negative,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),virginamerica really aggressive blast obnoxiou...,0,"[input_ids, token_type_ids, attention_mask]"
4,570300817074462722,negative,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),virginamerica really big bad thing,0,"[input_ids, token_type_ids, attention_mask]"


In [23]:
from transformers import BertTokenizer

class tokenizing:
    def __int__(
            self

    ):
        return "Hello"


In [15]:
import torch

class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.encodings['input_ids'])

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

In [32]:
from sklearn.model_selection import train_test_split

# Split the dataset
train_texts, temp_texts, train_labels, temp_labels = train_test_split(data['tokenized'].to_list(), data['label'].to_list(), test_size=0.3, random_state=42)
val_texts, test_texts, val_labels, test_labels = train_test_split(temp_texts, temp_labels, test_size=0.5, random_state=42)

In [30]:
def convert_to_dicts(tokenized_texts):
    input_ids = [d['input_ids'] for d in tokenized_texts]
    attention_masks = [d['attention_mask'] for d in tokenized_texts]
    return {'input_ids': input_ids, 'attention_mask': attention_masks}

# Convert to lists of dictionaries

def lists_to_dicts(train_texts, val_texts, test_texts):
    train_encodings = convert_to_dicts(train_texts)
    val_encodings = convert_to_dicts(val_texts)
    test_encodings = convert_to_dicts(test_texts)
    return train_encodings, val_encodings, test_encodings

# Create three dataset objects using the SentimentDataset
# train_dataset = SentimentDataset(train_encodings, train_labels)
# val_dataset = SentimentDataset(val_encodings, val_labels)
# test_dataset = SentimentDataset(test_encodings, test_labels)
def datasets_from_setimentdatasets(train_encodings, val_encodings, test_encodings):
    train_dataset = SentimentDataset(train_encodings, train_labels)
    val_dataset = SentimentDataset(val_encodings, val_labels)
    test_dataset = SentimentDataset(test_encodings, test_labels)
    return train_dataset, val_dataset, test_dataset

In [33]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

def claassification_from_pretrained():
    model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
    return model


# model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)